<h1>What to do:</h1>
<ul>
    <li>Вероятность ложной тревоги</li>
    <li>Вероятность пропуска обнаружения ошибки</li>
    <li>Вероятность суммарной ошибки классификации</li>
</ul>
<h2>Input:</h2>
<ul>
    <li>2 СВ с нормальным распределением</li>
    <li>2 вероятности отнесения каждой из 2 случайных величин к первому из 2 классов (сумма равна 1)</li>
</ul>

<h3>Imports</h3>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import gif
from IPython.display import Image
import random
from datetime import datetime
import collections.abc
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from sage.repl.ipython_kernel.interact import *
import ipywidgets as w
from scipy.stats import truncnorm
import scipy.stats as stats
from math import ceil

<h3>Constants</h3>

In [2]:
points = 100000;
start1 = 100; end1 = 740;
start2 = -100; end2 = 540;
coeff = 440000;
xOffset = 600;
width = 1000; height = 1000;

<h3>Core</h3>

In [4]:
def get_truncated_normal(mean, sd, low, upp):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def get_normalized_bins(bins, count):
    res = [0] * 102
    for i in range(len(count)):
        idx = ceil(bins[i])
        if 0 <= idx <= 100:
            res[idx] += count[i]
    return res

def update_plot(alpha):
    left = 0
    right = 100
    bot = 0
    top = 100
    stepCount = 200000
    x = np.linspace(left, right, stepCount)
    yd = 0 * x
    plt.plot(x, yd, linewidth=0)
    
    # generate points of means of each of 2 distributions, standart deviation
    mx1 = (right - left) * alpha / 2
    mx2 = ( (right - left) * (1 - alpha) / 2 ) + 2 * mx1
    
    sd1 = randint(10, 15)
    sd2 = randint(10, 15)
    
    g1 = get_truncated_normal(mean=mx1, sd=sd1, low=left, upp=right)
    g2 = get_truncated_normal(mean=mx2, sd=sd2, low=left, upp=right)
    
    # generate data according to 2 distributions
    
    
    data1 = g1.rvs(points)
    data2 = g2.rvs(points)
    
    

    count1, bins1, ignored1 = plt.hist(data1, 100, alpha=0.5, density=True);
    count2, bins2, ignored2 = plt.hist(data2, 100, alpha=0.5, density=True);

    
    nd1 = get_normalized_bins(bins1, count1)
    nd2 = get_normalized_bins(bins2, count2)
    idx = 0
 
    for i in range(20, 100):
        if nd1[i] < nd2[i]:
            idx = i
            break
    plt.axvline(idx)
    
    wa = 0
    wd = 0
    
    i = 0
    while true:
        if bins2[i] < idx:
            wa += count2[i]
        else:
            break
        i += 1

    i = 0    
    while true:
        if i == len(count1) or i == len(bins1):
            break
        if bins1[i] >= idx:
            wd += count1[i]
        i += 1
    
    if alpha == 1:
        alpha -= 1e-20
    if alpha == 0:
        alpha += 1e-20
    
    wa *= (1-alpha)
    wd *= alpha
    
    wt = wa + wd
    
    print(f"Wrong accept: {wa}")
    print(f"Wrong decline: {wd}")
    print(f"Wrong total: {wt}")
                                
        
    plt.axvline(mx1, color='b')
    plt.axvline(mx2, color='r')
    

def build_ui():
    sliders = [1]
    alpha = sliders[0] = w.FloatSlider(min=0, max=1, step=0.01, description='alpha')
    controls = w.VBox(sliders)
    plot = w.interactive_output(update_plot, {'alpha':alpha})
    ui = w.HBox([controls, plot])
    return ui

build_ui()